# AquaCrop-OSPy: Bridging the gap between research and practice in crop-water modelling


<a href="https://colab.research.google.com/github/thomasdkelly/aquacrop/blob/master/tutorials/AquaCrop_OSPy_Notebook_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>


This series of notebooks provides users with an introduction to AquaCrop-OSPy, an open-source Python implementation of the U.N. Food and Agriculture Organization (FAO) AquaCrop model. AquaCrop-OSPy is accompanied by a series of Jupyter notebooks, which guide users interactively through a range of common applications of the model. Only basic Python experience is required, and the notebooks can easily be extended and adapted by users for their own applications and needs.


This notebook series consists of four parts:

1.   <a href=https://colab.research.google.com/github/thomasdkelly/aquacrop/blob/master/tutorials/AquaCrop_OSPy_Notebook_1.ipynb>Running an AquaCrop-OSPy model</a>
2.   <a href=https://colab.research.google.com/github/thomasdkelly/aquacrop/blob/master/tutorials/AquaCrop_OSPy_Notebook_2.ipynb>Estimation of irrigation water demands</a>
3.   <a href=https://colab.research.google.com/github/thomasdkelly/aquacrop/blob/master/tutorials/AquaCrop_OSPy_Notebook_3.ipynb>Optimisation of irrigation management strategies</a>
4.  <a href=https://colab.research.google.com/github/thomasdkelly/aquacrop/blob/master/tutorials/AquaCrop_OSPy_Notebook_4.ipynb>Projection of climate change impacts</a>


# Notebook 1: Getting started: Running your first simulation with AquaCrop-OSPy



In this notebook, you will learn interactively how to setup and run you first AquaCrop-OSPy simulation. We begin by showing how to define key model input parameters and variables, and then show how to execute a model simulation and interpret output files.

The first of these notebooks outlines how to setup and run single and multi-season simulations for a selected cropping system. Examples are provided about how to setup and define relevant crop, soil, weather and management parameter values and inputs, with more advanced customization guides given in the Appendices A-D.

## Notes on Google Colab

If you are unfamiliar with Jupyter Notebooks or Google Colab, <a href="https://colab.research.google.com/notebooks/intro.ipynb">here</a> is an introductory notebook to get you started. In short, these are computable documents that let you combine text blocks such as these (including html, LaTeX, etc.) with code blocks that you can write, edit and execute. To run the code in each cell either click the run button in the cell's top left corner or hit SHIFT-ENTER when the cell is selected. You can also navigate through the document using the table of contents on the left hand side.

W recommend you save a copy of this notebook to your dive (File->Save a copy to drive). Then any changes you make to this notebook will be saved and you can open it it again at any time and carry on. 

<a id='Imports'><a/>

## Imports



In order to use AquaCrop-OSPy inside this notebook we first need to install and import it. Installing aquacrop is as simple as running `pip install aquacrop` or `!pip install aquacrop==VERSION` to install a specific version. In cell below we also use the `output.clear` function to keep everything tidy. 

In [ ]:
# !pip install aquacrop==0.2

# from google.colab import output
# output.clear()

In [ ]:
# only used for local development

# import sys
# _=[sys.path.append(i) for i in ['.', '..']]

Now that `aquacrop` is installed we need to import the various components into the notebook. All these functions do not have to be imported at once as we have done in the cell below but it will help keep things clearer throughout this notebook.

In [ ]:
# import aquacrop functions (the * simply means 'all')
from aquacrop.classes import    *
from aquacrop.core import       *

# Selecting Model Components

Running an AquaCrop-OSPy model requires the selection of 5 components:

1.   Daily climate measurements
3.   Soil selection
4.   Crop selection
5.   Initial water content
1.   Simulation start and end dates

We will go through the selection of these components in turn below.

## Climate Measurements

AquaCrop-OSPy requires weather data to be specified over the cropping period being simulated. This includes a daily time series of minimum and maximum temperatures [C], precipitation [mm], and reference crop evapotranspiration [mm].

To import these data into the model, a .txt file containing relevant weather data must be created by the user in the following space delimited format.

![picture](https://drive.google.com/uc?export=view&id=1WG0gN4fCYgQs-EQXhXUvNrPktFJOZut9)

If you are running this notebook locally you will need to specify the file path to the weather data file on your computer. If you are running the notebook via Google Colab you can upload the file through the tab on the left so that it is available in the current directory.

![picture](https://drive.google.com/uc?export=view&id=1xTg4b1W-Nvi8kuK3ytVybTZpaWSflnNE)

To load the weather data into the model, use the `prepare_weather` function, passing in the filepath of the .txt file. This `prepare_weather` function will create a pandas DataFrame in Python storing the imported weather data in the correct format for subsequent simulations.



In [ ]:
# specify filepath to weather file (either locally or imported to colab)
# filepath= 'YOUR_WEATHER_FILE.TXT'

# weather_data = prepare_weather(filepath)
# weather_data

AquaCrop-OSPy also contains a number of in-built example weather files. These can be accessed using the `get_filepath` function as shown below. Once run then use the `prep_weather` function as above to convert the data to a pandas DataFrame ready for use. A full list of the built-in weather files can be found in Appendix A.

In [ ]:
# locate built in weather file
filepath=get_filepath('tunis_climate.txt')

weather_data = prepare_weather(filepath)
weather_data

## Soil



Selecting a soil type for an AquaCrop-OSPy simulation is done via the `SoilClass` object. This object contains all the compositional and hydraulic properties needed for the simulation. The simplest way to select a `SoilClass` is to use one of the built-in soil types taken from AquaCrop defaults. A visual representation of these defaults is shown below.

![picture](https://drive.google.com/uc?export=view&id=11CDRTYgYrYxsQyEwih_0jDAozIu_l-ak)

As an example, to select a 'sandy loam' soil, run the cell below. Appendix B details how a user can edit any of these built-in soil types or create their own custom soil profile. 

In [ ]:
sandy_loam = SoilClass(soilType='SandyLoam')

## Crop

The crop type used in the simulation is selected in a similar way to soil via a `CropClass`. To select a `CropClass` you need to specify the crop type and planting date. Any of the built-in crop types (currently Maize, Wheat, Rice, Potato) can be selected by running the cell below. Appendix C details how a user can edit any of the built-in crop parameters or create custom crops. 

In [ ]:
wheat = CropClass('Wheat', PlantingDate='10/01')

## Initial water content

Specifying the intial soil-water content at the begining of the simulation is done via the `InitWCClass`. When creating an `InitWCClass`, it needs a list of locations and soil water contents. This table below details all the input paramaters for the `InitWCClass`.

Variable Name | Type | Description | Default
--- | --- | --- | ---
wc_type| `str` | Type of value | 'Prop'
|| 'Prop' = 'WP' / 'FC' / 'SAT' |
|| 'Num' = XXX m3/m3 |
|| 'Pct' = % TAW |
Method | `str` | 'Depth' = Interpolate depth points; 'Layer' = Constant value for each soil layer | 'Layer'
depth_layer| `list` | locations in soil profile (soil layer or depth) | [1]
value| `list` | value at that location | ['FC']

In the cell below we initialize the water content to be Field Capacity (FC) accross the whole soil profile. 


In [ ]:
InitWC = InitWCClass(value=['FC'])

## Model

Additional model components you can specify include irrigation management, field management and groundwater conditions. Details on how to specify these are detailed in Appendix D however they will both default to none if not specified. 

Once you have defined your weather data, `CropClass`,  `SoilClass` and `InitWCClass` then you're ready to run your simulation.


To run a simulation we need to combine the components we have selected into an `AquaCropModel`. It is here where we specify the simulation start date and end date (YYYY/MM/DD). Running the cell below will create an AquaCropModel simulation with all the parameters we have specified so far. 

In [ ]:
# combine into aquacrop model and specify start and end simulation date
model = AquaCropModel(SimStartTime=f'{1979}/10/01',
                      SimEndTime=f'{1985}/05/30',
                      wdf=weather_data,
                      Soil=sandy_loam,
                      Crop=wheat,
                      InitWC=InitWC)

Now first initialize the model using `.initialize()`. The model can then be run forwards N days using `.step(N)`. Most of the time you will want to instead run the model till the end of the simulation which is done by running `.step(till_termination=True)`


In [ ]:
# initilize model
model.initialize()
# run model till termination
model.step(till_termination=True)

Once the model has finished running, four different output files will be produced. The `Flux` output shows daily water flux variables such as total water storage. The `Water` output shows daily water storage in each compartment. The `Growth` output details daily crop variables such as canopy cover. The `Final` output lists the final Yield and total Irrigation for each season. These outputs can be accessed through the cell below. Use `.head(N)` to view the first `N` rows. Full details on the output files can be found in Appendix E.

In [ ]:
# model.Outputs.Flux.head()
# model.Outputs.Water.head()
# model.Outputs.Growth.head()
model.Outputs.Final.head()

Congratulations, you have run your first AquaCrop-OSPy model. As a final example, let's create and run another model with a different soil type and compare the results. 

In [ ]:
# combine into aquacrop model and specify start and end simulation date
model_clay = AquaCropModel(SimStartTime=f'{1979}/10/01',
                      SimEndTime=f'{1985}/05/30',
                      wdf=weather_data,
                      Soil=SoilClass('Clay'),
                      Crop=wheat,
                      InitWC=InitWC)

model_clay.initialize()
model_clay.step(till_termination=True)

Let's use the pandas library to collate our seasonal yields so we can visualize our results.

In [ ]:
import pandas as pd # import pandas library

names=['Sandy Loam','Clay']

#combine our two output files
dflist=[model.Outputs.Final,
        model_clay.Outputs.Final] 

outlist=[]
for i in range(len(dflist)): # go through our two output files
    temp = pd.DataFrame(dflist[i]['Yield (tonne/ha)']) # extract the seasonal yield data
    temp['label']=names[i] # add the soil type label
    outlist.append(temp) # save processed results

# combine results
all_outputs = pd.concat(outlist,axis=0)

Now we can leverage some of pythons great plotting libraries `seaborn` and `matplotlib` to visualize and compare the yields from the two different soil types.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#create figure
fig,ax=plt.subplots(1,1,figsize=(10,7),)

# create box plot
sns.boxplot(data=all_outputs,x='label',y='Yield (tonne/ha)',ax=ax,)

# labels and font sizes
ax.tick_params(labelsize=15)
ax.set_xlabel(' ')
ax.set_ylabel('Yield (tonne/ha)',fontsize=18)

And this brings an end to Notebook 1. Feel free to edit any of the code cells, import your own weather data, try new soils and crops. 

In Notebook 2 we use the model to estimate irrigation water demands.

# Appendix A: Built-in weather files

AquaCrop-OSPy includes a number of in-built weather files for different locations around the world, which users can select for model simulations. These have been taken from AquaCrop defaults (with the exception of Champion Nebraska **link**). In-built weather files currently include:


1.   Tunis (date) | 'tunis_climate.txt'
2.   Brussels (date) | 'brussels_climate.txt'
3.   Hyderabad (date) | 'weather_hyd.txt'
4.   Champion, Nebraska (date) | 'champion_weather.txt'

The filepath to these can be found using the `get_filepath` function and the data can be read in using the `prepare_weather` function.

In [ ]:
# get location of built in weather data file
path = get_filepath('hyderabad_climate.txt')
# read in weather data file and put into correct format
wdf = prepare_weather(path)
# show weather data
wdf.head()

# Appendix B: Custom Soils

Custom soil classes can be created by passing 'custom' as the soil type when creating a `SoilClass`. In the cell below we create a custom soil with a curve number (CN=46) and readily evaporable water (REW=7). Here is a full list of the parameters you can specify:

Variable Name | Description | Default
--- | --- | ---
soilType | Soil classification e.g. 'sandy_loam' | REQUIRED
dz | thickness of each soil compartment e.g. 12 compartments of thickness 0.1m | [0.1]*12
CalcSHP | Calculate soil hydraulic properties (0 = No, 1 = Yes) | 0
AdjREW | Adjust default value for readily evaporable water (0 = No, 1 = Yes) | 1
REW | Readily evaporable water (mm) | 9.0
CalcCN | Calculate curve number (0 = No, 1 = Yes) | 1
CN | Curve Number | 61.0
zRes | Depth of restrictive soil layer (negative value if not present) | -999
 | **The parameters below should not be changed without expert knowledge** | 
EvapZsurf | Thickness of soil surface skin evaporation layer (m) | 0.04
EvapZmin | Minimum thickness of full soil surface evaporation layer (m) | 0.15
EvapZmax | Maximum thickness of full soil surface evaporation layer (m) | 0.30
Kex | Maximum soil evaporation coefficient | 1.1
fevap | Shape factor describing reduction in soil evaporation in stage 2. | 4
fWrelExp | Proportional value of Wrel at which soil evaporation layer expands | 0.4
fwcc | Maximum coefficient for soil evaporation reduction due to sheltering effect of withered canopy | 50
zCN | Thickness of soil surface (m) used to calculate water content to adjust curve number | 0.3
zGerm | Thickness of soil surface (m) used to calculate water content for germination | 0.3
AdjCN | Adjust curve number for antecedent moisture content (0: No, 1: Yes) | 1
fshape_cr | Capillary rise shape factor | 16
zTop | Thickness of soil surface layer for water stress comparisons (m) | 0.1





In [ ]:
custom = SoilClass('custom',CN=46,REW=7)

Soil hydraulic properties are then specified using `.add_layer()`. This function needs the thickness of the soil layer [m] (just the depth of soil profile if only using 1 layer), the water content at Wilting Point [m^3/m^3], Field Capacity [m^3/m^3], Saturation [m^3/m^3], as well as the hydraulic conductivity [mm/day] and soil penetrability [%].

In [ ]:
custom.add_layer(thickness=custom.zSoil,thWP=0.24,
                 thFC=0.40,thS=0.50,Ksat=155,
                 penetrability=100)

Soil hydraulic properties can also be specified using the soil textural composition. This is done using the `.add_layer_from_texture()` function. This function needs the soil thickness [m], sand, clay, organic matter content [%], and the penetrability [%]

In [ ]:
custom = SoilClass('custom',CN=46,REW=7)

custom.add_layer_from_texture(thickness=custom.zSoil,
                              Sand=10,Clay=35,
                              OrgMat=2.5,penetrability=100)

To view your custom soil profile simple run the cell below.

In [ ]:
custom.profile

Both these layer creation methods can be combined together to create multi layered soils. 

In [ ]:
custom = SoilClass('custom',CN=46,REW=7)

custom.add_layer(thickness=0.3,thWP=0.24,
                 thFC=0.40,thS=0.50,Ksat=155,
                 penetrability=100)

custom.add_layer_from_texture(thickness=1.5,
                              Sand=10,Clay=35,
                              OrgMat=2.5,penetrability=100)

custom.profile

In AquaCrop-OSPy, as in AquaCrop and AquaCrop-OS, the soil is split into compartments. By default these are 12 compartments of thickness 0.1m where the bottom layers will expand in order to exceed the maximum crop root depth. 

This depth (`dz`) can be also be altered by the user by changing the `dz` argument. For example: Lets say we want the top 6 compartments to be 0.1m each and the bottom 6 compartments to be 0.2m each...

In [ ]:
sandy_loam = SoilClass('SandyLoam',dz=[0.1]*6+[0.2]*6)

Similarly default soil types can be adjust by passing in the changed variables.

In [ ]:
local_sandy_loam = SoilClass('SandyLoam',dz=[0.1]*6+[0.2]*6,CN=46,REW=7)
local_sandy_loam.profile.head()

# Apendix C: Custom Crops


It is more likely that the user will want to modify one of the built in crops (as opposed to modelling a brand new crop. To do this simply pass in the altered parameters when you create the `CropClass`. Any parameters you specify here will override the crop defaults. To model a crop that does not have built in defaults, you can specify the crop type 'custom' and pass in all the parameters listed in the table below.

In [ ]:
local_wheat = CropClass('Wheat',
                        PlantingDate='11/01',
                        HarvestDate='06/30',
                        CGC = 0.0051,CDC = 0.0035)

Below is a full list of the crop parameters that can be altered.

Variable Name | Default | Description
--- | --- | ---
Name | | Crop Name e.ge. 'maize'
CropType | | Crop Type (1 = Leafy vegetable, 2 = Root/tuber, 3 = Fruit/grain)
PlantMethod | | Planting method (0 = Transplanted, 1 =  Sown)
CalendarType | | Calendar Type (1 = Calendar days, 2 = Growing degree days)
SwitchGDD | | Convert calendar to GDD mode if inputs are given in calendar days (0 = No; 1 = Yes)
PlantingDate | | Planting Date (mm/dd)
HarvestDate | | Latest Harvest Date (mm/dd)
Emergence  | | Growing degree/Calendar days from sowing to emergence/transplant recovery
MaxRooting  | | Growing degree/Calendar days from sowing to maximum rooting
Senescence  | | Growing degree/Calendar days from sowing to senescence
Maturity  | | Growing degree/Calendar days from sowing to maturity
HIstart  | | Growing degree/Calendar days from sowing to start of yield formation
Flowering  | | Duration of flowering in growing degree/calendar days (-999 for non-fruit/grain crops)
YldForm  | | Duration of yield formation in growing degree/calendar days
GDDmethod  | | Growing degree day calculation method
Tbase  | | Base temperature (degC) below which growth does not progress
Tupp  | | Upper temperature (degC) above which crop development no longer increases
PolHeatStress  | | Pollination affected by heat stress (0 = No, 1 = Yes)
Tmax_up  | | Maximum air temperature (degC) above which pollination begins to fail
Tmax_lo  | | Maximum air temperature (degC) at which pollination completely fails
PolColdStress | | Pollination affected by cold stress (0 = No, 1 = Yes)
Tmin_up  | | Minimum air temperature (degC) below which pollination begins to fail
Tmin_lo  | | Minimum air temperature (degC) at which pollination completely fails
TrColdStress  | | Transpiration affected by cold temperature stress (0 = No, 1 = Yes)
GDD_up  | | Minimum growing degree days (degC/day) required for full crop transpiration potential
GDD_lo | | Growing degree days (degC/day) at which no crop transpiration occurs
Zmin  | | Minimum effective rooting depth (m)
Zmax  | | Maximum rooting depth (m)
fshape_r | | Shape factor describing root expansion
SxTopQ  | | Maximum root water extraction at top of the root zone (m3\/ m3\/ day)
SxBotQ  | | Maximum root water extraction at the bottom of the root zone (m3\/ m3\/ day)
SeedSize | | Soil surface area (cm2) covered by an individual seedling at 90% emergence
PlantPop | | Number of plants per hectare
CCx   | | Maximum canopy cover (fraction of soil cover)
CDC  | | Canopy decline coefficient (fraction per GDD/calendar day)
CGC | | Canopy growth coefficient (fraction per GDD)
Kcb  | | Crop coefficient when canopy growth is complete but prior to senescence
fage | |  Decline of crop coefficient due to ageing (%/day)
WP  | | Water productivity normalized for ET0 and C02 (g/m2)
WPy | | Adjustment of water productivity in yield formation stage (% of WP)
fsink  | | Crop performance under elevated atmospheric CO2 concentration (%/100)
HI0  | | Reference harvest index
dHI_pre  | | Possible increase of harvest index due to water stress before flowering (%)
a_HI  | | Coefficient describing positive impact on harvest index of restricted vegetative growth during yield formation
b_HI | | Coefficient describing negative impact on harvest index of stomatal closure during yield formation
dHI0 | | Maximum allowable increase of harvest index above reference value
Determinant  | | Crop Determinacy (0 = Indeterminant, 1 = Determinant)
exc  | | Excess of potential fruits
p_up1 | | Upper soil water depletion threshold for water stress effects on affect canopy expansion
p_up2  | | Upper soil water depletion threshold for water stress effects on canopy stomatal control
p_up3  | | Upper soil water depletion threshold for water stress effects on canopy senescence
p_up4  | | Upper soil water depletion threshold for water stress effects on canopy pollination
p_lo1  | | Lower soil water depletion threshold for water stress effects on canopy expansion
p_lo2 | | Lower soil water depletion threshold for water stress effects on canopy stomatal control
p_lo3 | |  Lower soil water depletion threshold for water stress effects on canopy senescence
p_lo4  | | Lower soil water depletion threshold for water stress effects on canopy pollination
fshape_w1  | | Shape factor describing water stress effects on canopy expansion
fshape_w2 | | Shape factor describing water stress effects on stomatal control
fshape_w3  | | Shape factor describing water stress effects on canopy senescence
fshape_w4 | | Shape factor describing water stress effects on pollination
 | | **The paramaters below should not be changed without expert knowledge** 
fshape_b | 13.8135 | Shape factor describing the reduction in biomass production for insufficient growing degree days
PctZmin | 70 | Initial percentage of minimum effective rooting depth
fshape_ex | -6 | Shape factor describing the effects of water stress on root expansion
ETadj | 1 | Adjustment to water stress thresholds depending on daily ET0 (0 | No, 1 | Yes)
Aer | 5 | Vol (%) below saturation at which stress begins to occur due to deficient aeration
LagAer | 3 | Number of days lag before aeration stress affects crop growth
beta | 12 | Reduction (%) to p_lo3 when early canopy senescence is triggered
a_Tr | 1 | Exponent parameter for adjustment of Kcx once senescence is triggered
GermThr | 0.2 | Proportion of total water storage needed for crop to germinate
CCmin | 0.05 | Minimum canopy size below which yield formation cannot occur
MaxFlowPct | 33.3 | Proportion of total flowering time (%) at which peak flowering occurs
HIini | 0.01 | Initial harvest index
bsted | 0.000138 | WP co2 adjustment parameter given by Steduto et al. 2007
bface | 0.001165 | WP co2 adjustment parameter given by FACE experiments



# Appendix D: Managment and initial conditions

Field management and groundwater conditions can also be specified in an AquaCrop-OSPy with a `FieldMngtClass` and `GWClass` respectively. If these are not specified then they will default to None. 

### Field management

Field Management is specified via the `FieldMngtClass`. These are largely based around the inclusion of either mulches or bunds. Two `FieldMngtClass` objects can be created for the fallow and growing period. The parameters you can specify in a `FieldMngtClass` are:

Variable Name | Type | Description | Default
--- | --- | --- | ---
Mulches| `bool` | Soil surface covered by mulches (True or False) | False
MulchPct| `float` |  Area of soil surface covered by mulches (%) | 50
fMulch| `float` |  Soil evaporation adjustment factor due to effect of mulches | 0.5
Bunds| `bool` |Surface bunds present (True or False) | False
zBund| `float` |  Bund height (m) | 0
BundWater| `float` |  Initial water height in surface bunds (mm) | 0.
CNadj| `bool` | field conditions affect curve number (True or False) | False
CNadjPct| `float` |   Change in curve number (positive or negative) (%) | 0
SRinhb| `bool` | Management practices fully inhibit surface runoff (True or False) | False


In the cell below we create an `AquaCropModel` passing in a `FieldMngtClass` with 100% field covering by mulches. 



In [ ]:
mulches_model = AquaCropModel(SimStartTime=f'{1979}/10/01',
                      SimEndTime=f'{1985}/05/30',
                      wdf=weather_data,
                      Soil=sandy_loam,
                      Crop=wheat,
                      InitWC=InitWC,
                      FieldMngt=FieldMngtClass(Mulches=True,
                                               MulchPct=100,
                                               fMulch=0.5))


In [ ]:
mulches_model.initialize()
mulches_model.step(till_termination=True)


In [ ]:
# get final output
mulches_model.Outputs.Final

### Groundwater

Finally we can take a look at how to specify groundwater depth. This is done via the `GWClass` which takes in the following parameters:

Variable Name | Type | Description | Default
--- | --- | --- | ---
WaterTable| `str` | water table considered 'Y' or 'N' | 'N'
Method | `str` | Water table input data = 'Constant' / 'Variable' | 'Constant'
dates| `list[str]` | water table observation dates 'YYYYMMDD' | []
values| `list[float]` | value at that location | []

The `GWClass` needs a list of dates and water table depths. If `Method='Variable'` the water table depth will be linearly interpolated between these dates. The cell below creates a model with a constant groundwater depth of 2m 


In [ ]:
# constant groundwater depth of 2m
gw_model = AquaCropModel(SimStartTime=f'{1979}/10/01',
                      SimEndTime=f'{1985}/05/30',
                      wdf=weather_data,
                      Soil=sandy_loam,
                      Crop=wheat,
                      InitWC=InitWC,
                      Groundwater=GwClass(WaterTable='Y',
                                       dates=[f'{1979}/10/01'],
                                       values=[2])
                    )

In [ ]:
gw_model.initialize()
gw_model.step(till_termination=True)

In [ ]:
gw_model.Outputs.Final

# Appendix E: Output files

There are 4 different outputs produced by the model:


1.  Daily Water Flux 

Variable Name | Unit
--- | --- 
water content | mm
groundwater depth | mm
surface storage | mm
irrigation |  mm
infiltration | mm
runoff | mm
deep percolation | mm
capillary rise | mm
groundwater inflow | mm
actual surface evaporation | mm
potential surface evaporation | mm
actual transpiration | mm
precipitation |  mm


2.   Soil-water content in each soil compartment

Variable Name | Unit
--- | --- 
compartment water content | mm/mm

3.   Crop growth

Variable Name | Unit
--- | --- 
growing degree days | -
cumulative growing degree days | -
root depth | m
canopy cover |  -
canopy cover (no stress) | -
biomass | kg/ha
biomass (no stress) | kg/ha
harvest index | -
adjusted harvest index | -
yield | t/ha

4.   Final summary (seasonal total)

Variable Name | Unit
--- | --- 
yield | t/ha
total irrigation | mm



Use the `.head(N)` command to view the first N entries of the output files below

In [ ]:
model.Outputs.Flux.head()
# model.Outputs.Water.head()
# model.Outputs.Growth.head(10)
# model.Outputs.Final.head()